# Derivatives

## 

In [1]:
import datetime as dt 
from datetime import date 
from contracts import * 

### A zero-coupon bond:

In [4]:
WHEN(
    dt.date(2024,2,28),
    SCALE(CONST(100), ONE(CAD))
)

WHEN(2024-02-28,SCALE(CONST(100),ONE(CAD)))

### A european call option.

In [2]:

option=WHEN(
    dt.date(2023,7,1),                            # on the 1st of JULY, acquire the right
    OR(                                           # to EITHER
        ZERO(),                                   # do nothing, OR
        AND(   
            SCALE(CONST(1),ONE(BTC)) ,            # acquire 1 BTC
            GIVE(                                 # for
                SCALE(CONST(30000),ONE(CAD))      # $30,000 CAD
            ),                                    
            
        )
    )
)

In [3]:
option

WHEN(2023-07-01,OR(ZERO(),AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(30000),ONE(CAD))))))

These are common instruments, so let's create some utility functions to create them:

In [4]:
def spottrade(left_amount:Decimal, left:Asset, right_amount:Decimal, right:Asset)->Contract:
    return AND(
        SCALE(CONST(left_amount),ONE(left)),
        GIVE(SCALE(CONST(right_amount),ONE(right)))
    )

def call(t:date, base:Asset, k:Decimal, quote:Asset)->Contract:
    'at time t acquire the right to purchase 1 unit of base at price k'
    return WHEN(
        t,
        OR(
            ZERO(),
            spottrade(1,base, k, quote)
        )
    )

def put(t:date,base:Asset,k:Decimal,quote:Asset)->Contract:
    'at time t acquire the right to sell 1 unit of base at price k'
    return WHEN(
        t,
        OR(
            ZERO(),
            spottrade(k,quote, 1, base)
        )
    )

def forward(t:date, contract:Contract)->Contract:
    'acquire the obligation to execute contract at time t '
    return WHEN(t,contract)


Agree to purchase 1 BTC @ 35,000 USD on December 31st.

In [5]:
f= forward(dt.date(2023,12,31),spottrade(1,BTC,35000,USD))

f

WHEN(2023-12-31,AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(35000),ONE(USD)))))

Acquire the right (but not the obligation!) to purchase 1 BTC @ 35,000 USD on December 31st

In [6]:
call(dt.date(2023,12,31),BTC,35000,USD)

WHEN(2023-12-31,OR(ZERO(),AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(35000),ONE(USD))))))

---

### Principal Protected Notes (PPN) 

Armed with these primitives, we can construct more complex products to produce payoff structures tuned to our clients needs. 

A common example is a Principal Protected Note: an instrument that allows you to participate in some of the possible upside of an asset while purchasing protection from price drops.

* maturity: June 1st 
* floor: 20k
* Ceiling: 30k

We do this by constructing three separate contracts and combining them:

We *buy* a forward contract on one unit of the underlying.
We *buy* a put to give us downside protection .
We *sell* a call with a higher strike price to make the overall instrument cheaper, at the cost of some upside participation.



In [7]:
t = dt.date(2023,7,1)


long_forward = WHEN(t, ONE(BTC))
long_put=put(t,BTC,Decimal("20"),USD)
short_call=GIVE(call(t, BTC, Decimal("20"), USD))
 

We then combine these into one contract:

In [8]:
ppn = AND(
    long_forward,
    AND(
        long_put, short_call
    )
)

In [9]:
ppn

AND(WHEN(2023-07-01,ONE(BTC)),AND(WHEN(2023-07-01,OR(ZERO(),AND(SCALE(CONST(20),ONE(USD)),GIVE(SCALE(CONST(1),ONE(BTC)))))),GIVE(WHEN(2023-07-01,OR(ZERO(),AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(20),ONE(USD)))))))))